In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.datasets
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [3]:

df_train = pd.read_csv('/content/drive/MyDrive/PG32 CS3244/smoteNCTrain.csv')
df_test = pd.read_csv('/content/drive/MyDrive/PG32 CS3244/smoteNCTest.csv')


In [4]:
y_train = df_train.iloc[:,-1] # labels

y_test = df_test.iloc[:,-1] # labels

df_total = df_train.append(df_test)

In [5]:
df_train.shape

(50164, 15)

In [6]:
X = df_total.iloc[:,1:-1] # all the variables
X_train = df_train.iloc[:,1:-1] # all the variables
y_train = df_train.iloc[:,-1] # labels

X_test = df_test.iloc[:,1:-1] # all the variables
y_test = df_test.iloc[:,-1] # labels

In [7]:
# one hot encoding
X_encoded = pd.get_dummies(X, columns=['FLAG_OWN_CAR', 
                                                   'FLAG_OWN_REALTY',
                                                   'NAME_INCOME_TYPE',
                                                   'NAME_EDUCATION_TYPE',
                                                   'NAME_FAMILY_STATUS',
                                                   'NAME_HOUSING_TYPE',
                                                   'FLAG_WORK_PHONE',
                                                   'OCCUPATION_TYPE',
                                                   ])
X_encoded

,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,FLAG_OWN_CAR_0,FLAG_OWN_CAR_1,FLAG_OWN_REALTY_0,FLAG_OWN_REALTY_1,NAME_INCOME_TYPE_0,...,OCCUPATION_TYPE_9,OCCUPATION_TYPE_10,OCCUPATION_TYPE_11,OCCUPATION_TYPE_12,OCCUPATION_TYPE_13,OCCUPATION_TYPE_14,OCCUPATION_TYPE_15,OCCUPATION_TYPE_16,OCCUPATION_TYPE_17,OCCUPATION_TYPE_18
0,0,135000.000000,-13566,-1900,2,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,315000.000000,-10328,-543,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,315000.000000,-18184,-3021,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,180000.000000,-13467,-3850,2,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,1,247500.000000,-13086,-1931,3,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21513,0,81000.000000,-23902,365243,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
21514,0,75177.780415,-15530,-451,2,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
21515,0,180000.000000,-10949,-1363,2,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
21516,0,202500.000000,-13881,-1157,2,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#scale

from sklearn.preprocessing import scale
X_scaled = scale(X_encoded)
X_train_scaled = X_scaled[0:50164]
X_test_scaled = X_scaled[50164:]

#print(X_train_encoded.shape) 
#print(X_test_encoded.shape)

In [9]:
X_train = X_train_scaled
X_test = X_test_scaled

In [10]:
X_scaled

array([[-0.52375168, -0.52269087,  0.60262107, ..., -0.11045491,
        -0.0498936 , -0.67359525],
       [-0.52375168,  1.32105267,  1.39899281, ..., -0.11045491,
        -0.0498936 , -0.67359525],
       [-0.52375168,  1.32105267, -0.53315556, ..., -0.11045491,
        -0.0498936 , -0.67359525],
       ...,
       [-0.52375168, -0.06175499,  1.24626062, ..., -0.11045491,
        -0.0498936 , -0.67359525],
       [-0.52375168,  0.16871295,  0.52514822, ..., -0.11045491,
        -0.0498936 , -0.67359525],
       [-0.52375168,  0.62964884,  0.05834353, ..., -0.11045491,
        -0.0498936 , -0.67359525]])

# MLP

In [11]:
# base line mlp

from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(50,100,150), max_iter=300,activation = 'logistic',solver='sgd',random_state=42)
model.fit(X_train, y_train)

y_predict = model.predict(X_test)
print(sum(y_predict == y_test))
print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print('Recall Score is {:.5}'.format(recall_score(y_test, y_predict))) # very bad recall...
print('Precision Score is {:.5}'.format(precision_score(y_test, y_predict)))
print('F1 Score is {:.5}'.format(f1_score(y_test, y_predict)))
print(pd.DataFrame(confusion_matrix(y_test,y_predict)))

11075
Accuracy Score is 0.51469
Recall Score is 0.97323
Precision Score is 0.50766
F1 Score is 0.66726
     0      1
0  604  10155
1  288  10471


In [12]:
X_train = X_train_scaled
X_test = X_test_scaled
#MLP with best set of parameters
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(30,20), max_iter=300,activation = 'relu',solver='sgd',random_state=42)
model.fit(X_train, y_train)

y_predict = model.predict(X_test)
print(sum(y_predict == y_test))
print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print('Recall Score is {:.5}'.format(recall_score(y_test, y_predict))) # very bad recall...
print('Precision Score is {:.5}'.format(precision_score(y_test, y_predict)))
print('F1 Score is {:.5}'.format(f1_score(y_test, y_predict)))
print(pd.DataFrame(confusion_matrix(y_test,y_predict)))

15249
Accuracy Score is 0.70866
Recall Score is 0.51919
Precision Score is 0.83598
F1 Score is 0.64056
      0     1
0  9663  1096
1  5173  5586


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


# tsne for linear separability

In [ ]:
from sklearn.manifold import TSNE
from keras.datasets import mnist
from sklearn.datasets import load_iris
from numpy import reshape
import seaborn as sns
import pandas as pd  

x = X_train
y = y_train
tsne = TSNE(n_components=2, verbose=1, random_state=42)
z = tsne.fit_transform(x) 

df = pd.DataFrame()
df["y"] = y
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 2),
                data=df).set(title="T-SNE projection") 

# SVM

In [15]:
from sklearn.svm import SVC
clf_svm = SVC(random_state=42,C=10,kernel='linear')
clf_svm.fit(X_train, y_train)

SVC(C=10, kernel='linear', random_state=42)

In [16]:
y_predict = clf_svm.predict(X_test_scaled)
print(sum(y_predict == y_test))
print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print('Recall Score is {:.5}'.format(recall_score(y_test, y_predict))) 
print('Precision Score is {:.5}'.format(precision_score(y_test, y_predict)))
print('F1 Score is {:.5}'.format(f1_score(y_test, y_predict)))

13863
Accuracy Score is 0.64425
Recall Score is 0.80946
Precision Score is 0.60843
F1 Score is 0.69469


In [17]:
from sklearn.svm import SVC
clf_svm = SVC(random_state=42,C=10,kernel='rbf',gamma=0.001)

clf_svm.fit(X_train, y_train)
y_predict = clf_svm.predict(X_test_scaled)
print(sum(y_predict == y_test))
print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print('Recall Score is {:.5}'.format(recall_score(y_test, y_predict))) 
print('Precision Score is {:.5}'.format(precision_score(y_test, y_predict)))
print('F1 Score is {:.5}'.format(f1_score(y_test, y_predict)))

14406
Accuracy Score is 0.66949
Recall Score is 0.69319
Precision Score is 0.66182
F1 Score is 0.67714


# Logistic Regression

In [13]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix
print("--------LR-------")
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42,C=1)
model.fit(X_train_scaled, y_train)


y_predict = model.predict(X_test_scaled)
print(sum(y_predict == y_test))
print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print('Recall Score is {:.5}'.format(recall_score(y_test, y_predict))) # very bad recall...
print('Precision Score is {:.5}'.format(precision_score(y_test, y_predict)))
print('F1 Score is {:.5}'.format(f1_score(y_test, y_predict)))

--------LR-------
13991
Accuracy Score is 0.6502
Recall Score is 0.74394
Precision Score is 0.62649
F1 Score is 0.68018


# DT

In [14]:
print("--------Decision Tree--------")
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state = 42,)

model.fit(X_train_scaled, y_train)

y_predict = model.predict(X_test_scaled)
print(sum(y_predict == y_test))
print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print('Recall Score is {:.5}'.format(recall_score(y_test, y_predict)))
print('Precision Score is {:.5}'.format(precision_score(y_test, y_predict)))
print('F1 Score is {:.5}'.format(f1_score(y_test, y_predict)))

--------Decision Tree--------
15387
Accuracy Score is 0.71508
Recall Score is 0.45032
Precision Score is 0.95713
F1 Score is 0.61248


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix

y_predict = clf_svm.predict(X_test_scaled)
print(sum(y_predict == y_test))
print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print('Recall Score is {:.5}'.format(recall_score(y_test, y_predict)))
print('Precision Score is {:.5}'.format(precision_score(y_test, y_predict)))
print('F1 Score is {:.5}'.format(f1_score(y_test, y_predict)))